<a href="https://colab.research.google.com/github/patrickpoirier51/linux/blob/master/Copy_of_openvino_21_2_darknet_yolo_model_converter_mask_nomask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert a Darknet YOLO model to BLOB (Mask - NoMask)

## Install OpenVINO 2021.2

In [ ]:
%%time
%%capture
## install tools. OpenVino takes some time to download: 10-15 min sometimes.
!sudo apt-get install -y pciutils cpio
!sudo apt autoremove
## download installation files
!wget https://registrationcenter-download.intel.com/akdlm/irc_nas/17504/l_openvino_toolkit_p_2021.2.185.tgz
path = "l_openvino_toolkit_p_2021.2.185.tgz"
## install openvino
!tar xf "{path}"

CPU times: user 197 ms, sys: 44.1 ms, total: 241 ms
Wall time: 30.4 s


In [ ]:
%%capture
%cd l_openvino_toolkit_p_2021.2.185/
!./install_openvino_dependencies.sh && \
    sed -i 's/decline/accept/g' silent.cfg && \
    ./install.sh --silent silent.cfg

In [ ]:
%%capture
## install prerequisites
!/opt/intel/openvino_2021.2.185/deployment_tools/model_optimizer/install_prerequisites/install_prerequisites.sh

### Check OpenVINO Installation

In [ ]:
!source /opt/intel/openvino_2021.2.185/bin/setupvars.sh && \
    /opt/intel/openvino_2021.2.185/deployment_tools/demo/demo_squeezenet_download_convert_run.sh

[setupvars.sh] OpenVINO environment initialized
target_precision = FP16
[setupvars.sh] OpenVINO environment initialized


###################################################



Installing dependencies
Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x

## Refer to the [this](https://docs.openvinotoolkit.org/2021.2/openvino_docs_MO_DG_prepare_model_convert_model_tf_specific_Convert_YOLO_From_Tensorflow.html) OpenVINO tutorial for this notebook.


## Clone Repository

We will use the Tensorflow implementation of Yolo in [this](https://github.com/mystic123/tensorflow-yolo-v3) repository and convert the yolo weights to tensorflow protobuf.

In [ ]:
%cd /content/
!git clone https://github.com/mystic123/tensorflow-yolo-v3

/content
Cloning into 'tensorflow-yolo-v3'...
remote: Enumerating objects: 219, done.
remote: Total 219 (delta 0), reused 0 (delta 0), pack-reused 219
Receiving objects: 100% (219/219), 57.63 KiB | 4.43 MiB/s, done.
Resolving deltas: 100% (112/112), done.


## Upload the YOLO Trained Model Weight File

After uploading,  make sure that the weight file is uploaded using the ls command. 

In [ ]:
!ls

l_openvino_toolkit_p_2021.2.185      tensorflow-yolo-v3
l_openvino_toolkit_p_2021.2.185.tgz  yolov3-tiny_mask_train_best.weights
sample_data


To confirm if the weights file was uploaded properly or not, compare the md5sum of the uploaded file and the original file. 

In [ ]:
!md5sum yolov3-tiny_mask_train_best.weights

e35bbf4387d641562f7897410add75b5  yolov3-tiny_mask_train_best.weights


## Create Labels File

In [ ]:
!echo "Mask" > class.names
!echo "No-mask" >> class.names

## Convert Model to Protobuf

In [ ]:
!python3 tensorflow-yolo-v3/convert_weights_pb.py --class_names class.names --data_format NHWC --weights_file yolov3-tiny_mask_train_best.weights --tiny --output_graph yolov3-tiny_mask_train_best.pb

## Convert Model to OpenVINO Intermediate Representation (IR)

In [ ]:
model_name = "yolov3-tiny_mask_train_best"

Use the openvino model optimizer to convert the YOLO model to IR. 


In [ ]:
#CONVERT MODEL to OPEN VINO IRv10.
%cd "/content/"
!source /opt/intel/openvino_2021.2.185/bin/setupvars.sh && \
    python /opt/intel/openvino_2021.2.185/deployment_tools/model_optimizer/mo_tf.py \
      --batch 1 \
      --data_type FP16 \
      --reverse_input_channels \
      --transformations_config /opt/intel/openvino_2021.2.185/deployment_tools/model_optimizer/extensions/front/tf/yolo_v3_tiny.json \
      --input_model={model_name}.pb

/content
[setupvars.sh] OpenVINO environment initialized
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/content/yolov3-tiny_mask_train_best.pb
	- Path for generated IR: 	/content/.
	- IR output name: 	yolov3-tiny_mask_train_best
	- Log level: 	ERROR
	- Batch: 	1
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	True
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementation: 	None
	- Update the configuration file with input/output node n

In [ ]:
#check directory containing the exported IR model
%ls 

class.names                          yolov3-tiny_mask_train_best.bin
l_openvino_toolkit_p_2021.2.185/     yolov3-tiny_mask_train_best.mapping
l_openvino_toolkit_p_2021.2.185.tgz  yolov3-tiny_mask_train_best.pb
sample_data/                         yolov3-tiny_mask_train_best.weights
tensorflow-yolo-v3/                  yolov3-tiny_mask_train_best.xml


## Compile the IR model to a `.blob` to use on DepthAI Modules / Platform

### We save the blob in the IR directory from above, corresponding to --output_dir parameter above. 

In [ ]:
!source /opt/intel/openvino_2021.2.185/bin/setupvars.sh && \
/opt/intel/openvino_2021.2.185/deployment_tools/inference_engine/lib/intel64/myriad_compile -m ./{model_name}.xml -o ./{model_name}.blob -ip U8 -VPU_NUMBER_OF_SHAVES 6 -VPU_NUMBER_OF_CMX_SLICES 6

[setupvars.sh] OpenVINO environment initialized
Inference Engine: 
	API version ............ 2.1
	Build .................. 2021.2.0-1877-176bdf51370-releases/2021/2
	Description ....... API
[Warning][VPU][Config] Deprecated option was used : VPU_MYRIAD_PLATFORM
Done


### Download the `.blob` File and Run It on the DepthAI Module / Platform

To download locally, use the file explorer on the left to locate the file in the --output_dir folder, then right click download. Colab takes a few seconds to prepare the file, then the download prompt will appear.